In [1]:
import numpy as np
import pandas as pd
import sys
import json
import math
from getcasedata import get_data

In [2]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w
data = get_data(False)
data.keys()
cases_np = None
log_cases_np = None
log_diff_cases_np = None
diff_cases_np = None
for state in data.keys():
    data[state] = np.concatenate((data[state][:6], moving_average(data[state], 7)))
    diff_state_np = np.diff(np.append(np.array([0], dtype=np.float32), data[state]))
    log_state_np = np.log(np.clip(data[state], 1, None))
    log_diff_state_np = np.diff(np.append(np.array([0], dtype=np.float32), log_state_np))
    if cases_np is None:
        cases_np = data[state]
        diff_cases_np = diff_state_np
        log_cases_np = log_state_np
        log_diff_cases_np = log_diff_state_np
    else:
        cases_np = np.concatenate((cases_np, data[state]))
        diff_cases_np = np.concatenate((diff_cases_np, diff_state_np))
        log_cases_np = np.concatenate((log_cases_np, log_state_np))
        log_diff_cases_np = np.concatenate((log_diff_cases_np, log_diff_state_np))

0
60941.0


/media/anli5005/HOMEDISK/Research/COVID-19/control/getstatedata2.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = np.diff(np.append(np.array([0], dtype=np.float32), data[data["state"] == state][data["date"] >= "2020-03-01"]["cases"].to_numpy(dtype=np.float32)))


In [3]:
abbr_df = pd.read_csv("state-abbreviations.csv") # Columns Code and State - maps state codes to state names

In [4]:
print(cases_np.shape)
print(cases_np[220:300])

(15889,)
[5.62857143e+02 5.80142857e+02 5.87142857e+02 5.80428571e+02
 5.71428571e+02 5.80428571e+02 6.59571429e+02 6.52571429e+02
 6.58714286e+02 6.70857143e+02 6.90142857e+02 6.96571429e+02
 7.22571429e+02 6.47714286e+02 6.50857143e+02 6.46571429e+02
 6.65142857e+02 6.74000000e+02 6.95285714e+02 7.24285714e+02
 7.31571429e+02 7.48714286e+02 7.69142857e+02 8.14857143e+02
 8.29714286e+02 8.34142857e+02 9.13428571e+02 9.48857143e+02
 1.03228571e+03 1.10157143e+03 1.17571429e+03 1.23428571e+03
 1.31442857e+03 1.39614286e+03 1.49757143e+03 1.39928571e+03
 1.60185714e+03 1.72285714e+03 1.77028571e+03 1.86500000e+03
 2.00600000e+03 2.11885714e+03 2.23214286e+03 2.18514286e+03
 2.15257143e+03 2.15414286e+03 2.04828571e+03 2.39057143e+03
 2.47300000e+03 2.67085714e+03 2.42042857e+03 2.56842857e+03
 2.57785714e+03 2.66142857e+03 2.40557143e+03 2.39071429e+03
 2.38885714e+03 2.60200000e+03 2.50400000e+03 2.47757143e+03
 2.45314286e+03 2.75614286e+03 2.83128571e+03 2.86342857e+03
 3.00771429e+03

In [5]:
tsa_df = pd.read_csv("tsa.csv")
tsa = []
for idx, row in iter(tsa_df.iterrows()):
    tsa.append(int(row["year"].replace(",", "")))
tsa = np.array(list(reversed(tsa)), dtype=np.int64)

In [6]:
svi = pd.read_csv("svi.csv")
svis = {}
min_svi = 100000
max_svi = 0
for i, row in svi.iterrows():
    state = row["LOCATION"].split(", ")[1]
    if state not in svis:
        svis[state] = []
    if not math.isnan(row["SPL_THEMES"]):
        s = row["SPL_THEMES"]
        if s != -999:
            svis[state].append(s)
    if i % 10000 == 0:
        print("Processed %d" % i)
for key in svis.keys():
    svis[key] = np.mean(np.array(svis[key]))

Processed 0


In [7]:
fips = pd.read_csv("cbg_fips_codes.csv")
states = {}
state_names = fips["state"].unique()
state_fips = fips["state_fips"].unique()
for n, f in zip(state_names, state_fips):
    states[f] = n
pop = pd.read_csv("cbg_b00.csv")
pops = {}
for i, row in pop.iterrows():
    state = states[int(str(row["census_block_group"])[:-12])]
    if state not in pops:
        pops[state] = 0
    if not math.isnan(row["B00001e1"]):
        pops[state] += row.astype("int")["B00001e1"]
    if i % 10000 == 0:
        print("Processed %d" % i)

Processed 0
Processed 10000
Processed 20000
Processed 30000
Processed 40000
Processed 50000
Processed 60000
Processed 70000
Processed 80000
Processed 90000
Processed 100000
Processed 110000
Processed 120000
Processed 130000
Processed 140000
Processed 150000
Processed 160000
Processed 170000
Processed 180000
Processed 190000
Processed 200000
Processed 210000
Processed 220000


In [8]:
area = pd.read_csv("area.csv")
pop_dens = {}
for k in pops.keys():
    try:
        name = next(iter(abbr_df[abbr_df["Code"] == k].iterrows()))[1]["State"]
        pop_dens[name] = pops[k] / int(next(iter(area[area["State"] == name].iterrows()))[1]["Land Area"].replace(",", ""))
    except StopIteration:
        continue

In [9]:
temp_df = pd.read_csv("temperature-all-states.csv")
temp = {}
for idx, row in iter(temp_df.iterrows()):
    if row["date"] >= "2020-03-01":
        state = next(iter(abbr_df[abbr_df["Code"] == row["state_code"]].iterrows()))[1]["State"]
        if state not in temp:
            temp[state] = []
        temp[state].append(row["temperature"])

In [10]:
flu_df = pd.read_csv("flu-season-all-states.csv")
flu = {}
for idx, row in iter(flu_df.iterrows()):
    if row["date"] >= "2020-03-01":
        state = next(iter(abbr_df[abbr_df["Code"] == row["state_code"]].iterrows()))[1]["State"]
        if state not in flu:
            flu[state] = []
        flu[state].append(row["season"])

In [11]:
social_dist_df = pd.read_csv("social-distance-all-states.csv")
social_dist = {}
for idx, row in iter(social_dist_df.iterrows()):
    if row["date"] >= "2020-03-01":
        state = next(iter(abbr_df[abbr_df["Code"] == row["state_code"]].iterrows()))[1]["State"]
        if state not in social_dist:
            social_dist[state] = []
        social_dist[state].append(row["soc_dist"])

In [12]:
stay_home_df = pd.read_csv("stay-home-all-states.csv")
stay_home = {}
for idx, row in iter(stay_home_df.iterrows()):
    if row["date"] >= "2020-03-01":
        state = next(iter(abbr_df[abbr_df["Code"] == row["state_code"]].iterrows()))[1]["State"]
        if state not in stay_home:
            stay_home[state] = []
        stay_home[state].append(row["change"])

In [13]:
foot_traffic_df = pd.read_csv("foot-traffic-all-states.csv")
foot_traffic = {}
for idx, row in iter(foot_traffic_df.iterrows()):
    if row["date"] >= "2020-03-01":
        state = next(iter(abbr_df[abbr_df["Code"] == row["state_code"]].iterrows()))[1]["State"]
        if state not in foot_traffic:
            foot_traffic[state] = []
        foot_traffic[state].append(row["change"])

In [14]:
hospitalized_df = pd.read_csv("all-states-history.csv")
hospitalized = {}
for idx, row in iter(hospitalized_df.iterrows()):
    if row["date"] >= "2020-03-01":
        try:
            state = next(iter(abbr_df[abbr_df["Code"] == row["state"]].iterrows()))[1]["State"]
        except StopIteration:
            continue
        if state not in hospitalized:
            hospitalized[state] = []
        hospitalized[state].append(row["hospitalizedIncrease"])
for state in hospitalized.keys():
    hospitalized[state] = list(reversed(hospitalized[state]))

In [15]:
tests_df = pd.read_csv("tests-all-states.csv")
tests = {}
for idx, row in iter(tests_df.iterrows()):
    if row["date"] >= "2020-03-01":
        state = next(iter(abbr_df[abbr_df["Code"] == row["state_code"]].iterrows()))[1]["State"]
        if state not in tests:
            tests[state] = []
        tests[state].append(row["tests"])

In [16]:
sg_file = open("safegraph/sums.json")
sg_raw = json.load(sg_file)
sg_file.close()
sg = [{}, {}, {}, {}, {}, {}]
for typ in range(len(sg_raw)):
    for key in sg_raw[typ].keys():
        try:
            state = next(iter(abbr_df[abbr_df["Code"] == key].iterrows()))[1]["State"]
            sg[typ][state] = sg_raw[typ][key][60:]
        except StopIteration:
            continue

In [26]:
case_buckets = np.percentile(cases_np, list(range(5, 100, 5)))
log_case_buckets = np.percentile(log_cases_np, [20, 40, 60, 80])

In [27]:
print(case_buckets)
print(log_case_buckets)

[1.77142857e+00 8.85714286e+00 2.54285714e+01 4.83714286e+01
 7.94285714e+01 1.15571429e+02 1.65114286e+02 2.42714286e+02
 3.28714286e+02 4.17285714e+02 5.32000000e+02 6.42771429e+02
 7.51457143e+02 8.84571429e+02 1.08214286e+03 1.39417143e+03
 1.90525714e+03 2.71788571e+03 4.73497143e+03]
[3.87890827 5.49188497 6.46578915 7.24005555]


In [19]:
def diff_buckets(arr):
    print(np.where(arr < 0))
    return np.concatenate((np.percentile(arr[np.where(arr < 0)], [33, 67]), np.array([0], dtype=np.float32), np.percentile(arr[np.where(arr > 0)], [33, 67])))

In [20]:
diff_case_buckets = diff_buckets(diff_cases_np)
log_diff_case_buckets = diff_buckets(log_diff_cases_np)
print(diff_case_buckets.shape)
print(diff_case_buckets)

(array([    1,     5,    24, ..., 15882, 15883, 15884]),)
(array([    1,     5,    24, ..., 15829, 15831, 15840]),)
(5,)
[-26.28571429  -4.71428571   0.           5.71428571  30.28571429]


In [21]:
df = pd.DataFrame(columns=["date", "state", "cases", "log_cases", "diff_cases", "diff_log_cases", "class", "log_class", "diff_class", "diff_log_class", "density", "svi", "tsa", "visits_hotels", "visits_restaurants", "visits_retail", "visits_personal_care", "visits_gyms", "visits_health", "temperature", "flu", "soc_dist", "tests", "hospitalized"])
row = 0
for state in data.keys():
    if state not in svis or state not in pop_dens or state not in sg[0] or state not in sg[1] or state not in sg[2] or state not in sg[3] or state not in sg[4]:
        continue
    d = data[state]
    for i in range(0, d.shape[0]):
        log = math.log(max(1, d[i]))
        diff = d[i]
        diff_log = log
        if i > 0:
            diff -= d[i - 1]
            diff_log -= math.log(max(1, d[i - 1]))
        df.loc[row] = [
            i,
            state,
            d[i],
            log,
            diff,
            diff_log,
            next(filter(lambda j: case_buckets[j] > d[i] if j < len(case_buckets) else True, range(len(case_buckets) + 1))),
            next(filter(lambda j: log_case_buckets[j] > log if j < len(log_case_buckets) else True, range(len(log_case_buckets) + 1))),
            next(filter(lambda j: diff_case_buckets[j] > diff if j < len(diff_case_buckets) else True, range(len(diff_case_buckets) + 1))),
            next(filter(lambda j: log_diff_case_buckets[j] > diff_log if j < len(log_diff_case_buckets) else True, range(len(log_diff_case_buckets) + 1))),
            pop_dens[state],
            svis[state],
            tsa[i],
            sg[0][state][i],
            sg[1][state][i],
            sg[2][state][i],
            sg[3][state][i],
            sg[4][state][i],
            sg[5][state][i],
            temp[state][i],
            flu[state][i],
            social_dist[state][i],
            tests[state][i],
            hospitalized[state][i]
        ]
        row += 1
    if row % 1 == 0:
        print("Done with row %d" % row)

Done with row 295
Done with row 590
Done with row 885
Done with row 1180
Done with row 1475
Done with row 1770
Done with row 2065
Done with row 2360
Done with row 2655
Done with row 2950
Done with row 3245
Done with row 3540
Done with row 3835
Done with row 4129
Done with row 4423
Done with row 4716
Done with row 5008
Done with row 5299
Done with row 5590
Done with row 5881
Done with row 6172
Done with row 6462
Done with row 6752
Done with row 7042
Done with row 7332
Done with row 7622
Done with row 7912
Done with row 8202
Done with row 8491
Done with row 8780
Done with row 9069
Done with row 9358
Done with row 9647
Done with row 9935
Done with row 10223
Done with row 10510
Done with row 10797
Done with row 11083
Done with row 11369
Done with row 11654
Done with row 11939
Done with row 12224
Done with row 12509
Done with row 12794
Done with row 13079
Done with row 13363
Done with row 13647
Done with row 13930
Done with row 14213
Done with row 14496
Done with row 14775


In [22]:
df.to_csv("data.csv")